In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import pandas as pd
from tqdm import tqdm
import os
from torchvision import models
from PIL import Image
import matplotlib.pyplot as plt
import torch
import torchvision.transforms as T

# shit EDA and data engineering shit

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
WORKING_DIR = '/content/drive/MyDrive/find_polar_bear'

# Setting up data

In [4]:
!pip install catalyst
!pip install albumentations --user
!pip show catalyst
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 446 kB 27.5 MB/s 
     |████████████████████████████████| 125 kB 68.9 MB/s 
     |████████████████████████████████| 143 kB 71.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Name: catalyst
Version: 22.4
Summary: Catalyst. Accelerated deep learning R&D with PyTorch.
Home-page: https://github.com/catalyst-team/catalyst
Author: Sergey Kolesnikov
Author-email: scitator@gmail.com
License: Apache License 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: torch, numpy, tensorboardX, accelerate, tqdm, hydra-slayer
Required-by: 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 28.5 MB/s 


In [5]:
import os
import cv2
import collections
import time 
import tqdm
from PIL import Image
from functools import partial
train_on_gpu = True

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

import torchvision
from torchvision import datasets, models, transforms
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

# import albumentations as albu
# from albumentations import torch as AT

# import catalyst
# from catalyst.data import Augmentor
# from catalyst.dl import utils
# from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
from catalyst.runners import SupervisedRunner
# from catalyst.contrib.models.segmentation import Unet
from catalyst.callbacks import DiceCallback, EarlyStoppingCallback, CheckpointCallback

#import segmentation_models_pytorch as smp


import os
import gc
import cv2
import time
import tqdm
import random
import collections
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
from functools import partial
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score


import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau

In [6]:
class CloudDataset(Dataset):
    def __init__(self, img_ids, datatype: str = 'train',
                 transforms = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Resize((256, 256)),
                                transforms.ToTensor()]),
                preprocessing=None):
        
        self.img_ids = img_ids
        self.transforms = transforms
        self.preprocessing = preprocessing

    def __getitem__(self, idx):
        img = cv2.imread(os.path.join(WORKING_DIR, f'bears_cropped/bears/{idx}.jpg'))
        try:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        except:
            raise IndexError
        mask = cv2.imread(os.path.join(WORKING_DIR, f'bears_cropped/masks/{idx}.png'))
        img = self.transforms(img)
        mask = self.transforms(mask)[:1, :, :]
        mask = (mask > 0.5).float()
        if self.preprocessing:
            preprocessed = self.preprocessing(image=img, mask=mask)
            img = preprocessed['image']
            mask = preprocessed['mask']
        return img, mask

    def __len__(self):
        return len(self.img_ids)

In [7]:
class double_conv(nn.Module):
    """(conv => BN => ReLU) * 2"""

    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        x = self.conv(x)
        return x


class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x):
        x = self.conv(x)
        return x


class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(nn.MaxPool2d(2), double_conv(in_ch, out_ch))

    def forward(self, x):
        x = self.mpconv(x)
        return x


class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()

        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode="bilinear", align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch // 2, in_ch // 2, 2, stride=2)

        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)

        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, (diffX // 2, diffX - diffX // 2, diffY // 2, diffY - diffY // 2))
        
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x


class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256, False)
        self.up2 = up(512, 128, False)
        self.up3 = up(256, 64, False)
        self.up4 = up(128, 64, False)
        self.outc = outconv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return torch.sigmoid(x)

In [11]:
model = UNet(n_channels=3, n_classes=1).float()
train_on_gpu = True
if train_on_gpu:
    model.cuda()

In [12]:
model.load_state_dict(torch.load(os.path.join(WORKING_DIR, 'model2.pt'), map_location=torch.device('cuda') if train_on_gpu else torch.device('cpu')))

<All keys matched successfully>

# training 

In [13]:
def find_bears(img):
    if torch.is_tensor(img):
        img = img.numpy()
    elif not isinstance(img, np.ndarray):
        raise Exception('Img is not numpy and is not torch.tensor')
    img = img.copy()
    img = (img * 255).astype(dtype=int)
    img[np.where(img > 200)] = 255
    img[np.where(img <= 200)] = 0
    img = img.reshape([256, 256])
    img = img.astype(dtype=np.uint8)

    centers = []
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for contour in contours:
        M = cv2.moments(contour)
        if M['m00'] != 0:
            cx = int(M['m10'] / M['m00'])
            cy = int(M['m01'] / M['m00'])
            centers.append([cx, cy])
    return centers

In [18]:
img_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor()])

def get_preds(img):
    SIDE = 256
    h, w, _ = img.shape
    all_bears = []
    for x in range(0, w, SIDE):
        for y in range(0, h, SIDE):
            x_dif = 0
            y_dif = 0
            if x + SIDE > w:
                x_dif = x + SIDE - w
            if y + SIDE > h:
                y_dif = y + SIDE - h
            part_img = img[y - y_dif:y+SIDE, x - x_dif:x+SIDE]
            part_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            data = img_transforms(part_img)
            data = torch.FloatTensor([data.numpy().tolist()])
            if train_on_gpu:
                data = data.cuda()
            output = ((model(data))[0]).cpu().detach()
            bears = find_bears(output)
            for i in range(len(bears)):
                bears[i][0] += x - x_dif
                bears[i][1] += y - y_dif
            all_bears.extend(bears)
    return all_bears

In [ ]:
df = pd.DataFrame([], columns=['name', 'x', 'y'])
for path in os.listdir(os.path.join(WORKING_DIR, 'arctic_with_bears/bears')):
    img = cv2.imread(os.path.join(os.path.join(WORKING_DIR, 'arctic_with_bears/bears'), path))
    preds = get_preds(img)
    for pred in preds:
        df.loc[len(df)] = [path, int(pred[0]), int(pred[1])]

In [20]:
df.to_csv('out.csv', index=False)